In [1]:
!pip install langchain == 0.3.24 -q
!pip install langchain-openai -q
!pip install langchain_chroma -q
!pip install langchain_community -q
!pip install langgraph -q
!pip install pypdf -q

ERROR: Invalid requirement: '==': Expected package name at the start of dependency specifier
    ==
    ^
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.

In [2]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [4]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('API_KEY')

In [5]:
llm = ChatOpenAI(model='gpt-4o', temperature=0.)

In [6]:
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small'
)

In [ ]:
pdf_loader = PyODFLoader("path/to/pdf")

In [ ]:
pages = pdf_loader.load()

In [ ]:
pages #load pages and view

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

In [ ]:
pages_split = text_splitter.split_documents(pages) #get splitted data

In [ ]:
pages_split
len(pages_split) #view chunks and how many chunks

In [ ]:
#store vectores (create a folder as vectorstore in directory and store)
vectorstore = Chroma.from_documents(
    documents=pages_split,
    embedding=embeddings,
    persist_directory='./vectorstore'
    collection_name="data"
)

In [ ]:
 #create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3} #according to prompt retrieve 3 most relevant chunks
)

In [ ]:
#retrieve wanted data
@tool
def retriever_tool(query:str) -> str:
  """This tool searches and returns the information about codepro.lk"""
  docs=retriever.invoke(query)
  if not docs:
    return "I found no relevant information"
  results = []
  for i,doc in enumerate(docs):
    results.append(f"Document {i+1}:\n\n{doc.page_content}")
  return "\n\n".join(results) #got data join and give as a single result

In [ ]:
tools = [retriever_tool]
llm_with_tools = llm.bind_tools(tools)
#we can compine several tools like wikipedia,google search to increase performance

In [ ]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
#retrieve data through this and decide if retrieve or not
def should_continue(state:AgentState):
  """Check if the last message contains tool calls."""
  result = staet["messages"][-1]
  return hasattr(result,'tool_calls') and len(result.tool_calls) >0

In [ ]:
system_prompt = """
You are an intelligent AI assistant who answers questions about codeprolk based
onthe PDF document loaded into your knowledge base.Use the retriever tool
available to answer questions about the given data. you can make multiple calls
if needed. If you need to look up some information before asking a follow up
question ,you are allowed to do that!.
Please always cite the specific parts of the documents you use to answers."""

In [ ]:
#if we have multiple tools we can create dictionary according to tools we have and mapping we having tools
tools_dict ={our_tool.name:our_tool for our_tool in tools}

In [ ]:
#LLM agent
def call_llm(state:AgentState) -> AgentState:
  """Function to call the LLM with the current state."""
  messages = list(state['messages'])
  messages = [SystemMessage(content=system_prompt)] + messages
  message = llm_with_tools.invoke(messages)
  print(message)
  return{'messages' : [message]}
  #returning resposnses as messages getting through LLM

In [ ]:
# Retriever Agent
def take_action(state: AgentState) -> AgentState:
    """Execute tool calls from the LLM's response."""

    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling Tool: {t['name']} with query: {t['args'].get('query', 'No Query')}")

        if not t['name'] in tools_dict:  # Checks if a valid tool is present
            print(f"\nTool: {t['name']} does not exist.")
            result = "Incorrect Tool Name, Please Retry and Select tool from List of Available Tools"
        else:
            result = tools_dict[t['name']].invoke(t['args'].get('query', ''))
            print(f"Result length: {len(str(result))}")

        # Appends the tool message
        results.append(ToolMessage(
            tool_call_id=t['id'],
            name=t['name'],
            content=str(result),   # content should contain result, not str
            output=str(result)
        ))

    print("Tools Execution Complete. Back to the Model!")
    return {'messages': results}



In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("llm",call_llm)
workflow.add_node("retriever_agent",take_action)

workflow.add_conditional_edges("llm",should_continue,{
    True:"retriever_agent",
    False:END
})
workflow.add_edge("retriever_agent","llm")
workflow.set_entry_point("llm") #we can use startpoint end point instead start and end

app = workflow.compile()

In [ ]:
from IPython.display import Image,display
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({"messages" : [HumanMessage(content = "What are the courses offer by codeprolk")]})

In [ ]:
result['messages'][-1].content

In [ ]:
result = app.invoke({"messages" : [HumanMessage(content = "What is blockchain")]})
#trying to call but do not have in the document